# Idea:
Our solution: LDA + keywords from clusters of BERT based embeddings of noun phrases and verbs :
- Each noun phrase and verb in the texts is  transformed to embedding vector using Universal Sentence Encoder (transformer based on BERT)
- Embedding vectors from (a) are clustered (HDBSCAN + UNET)
- Words/phrases with embedding vectors closest to the centers of resulting clusters form key word/phrase
- Each text in the training sample is converted to collection of key-phrases by replacing its noun phrases and verbs with keyword/phrases and deleting other words
- LDA is performed on the transformed texts


**Reference:**<br>
- Daniel Cer, Yinfei Yang, Sheng-yi Kong, Nan Hua, Nicole Limtiaco, Rhomni St. John, Noah Constant, Mario Guajardo-Céspedes, Steve Yuan, Chris Tar, Yun-Hsuan Sung, Brian Strope, Ray Kurzweil. **Universal Sentence Encoder.** *arXiv:1803.11175, 2018.*
- McInnes, L, Healy, J, **UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction**, *ArXiv e-prints 1802.03426, 2018*

# Load data and python libraries

In [1]:
# data processing libraries
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# display wider columns in pandas data frames where necessary
pd.set_option('max_colwidth',150)

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

import tensorflow_hub as hub
#Load the Universal Sentence Encoder's TF Hub module
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
model = hub.load(module_url)
print ("module %s loaded" % module_url)

import umap
import hdbscan

import pickle

TensorFlow version: 2.2.0
module https://tfhub.dev/google/universal-sentence-encoder-large/5 loaded


In [2]:
df_train = pd.read_csv("./data/train_grouped.tsv", sep="\t")
print("df_train.shape:", df_train.shape)
print("df_train.shape:",df_train.columns)

df_train.shape: (33982, 16)
df_train.shape: Index(['date', 'author', 'title', 'url', 'section', 'publication',
       'first_10_sents', 'list_of_first_10_sents', 'list_of_verb_lemmas',
       'noun_phrases', 'list_of_nouns', 'list_of_lemmas', 'ID',
       'group_level_1', 'group_level_2', 'group_level_3'],
      dtype='object')


# Getting text clusters through sentence embedding comparison

In [3]:
def get_embeddings(input):
    return model(input)

In [4]:
def get_word_embeddings(df_data, column = "word", N_batches=1):
    #split data into N batches
    N = N_batches

    part = int(len(df_data)/N)
    print(N, "batches with", part + 1, column + "s each")

    #get embeddings for each N words
    index = 0
    batch_num = 0
    list_dfs = []

    while index < len(df_data): 
        df_tmp = df_data.iloc[index : index + part].copy()
        df_tmp = df_tmp.reset_index(drop=True)
        print ("Batch number:", batch_num + 1, "out of ", N)

        df_batch_embeddings = pd.DataFrame(get_embeddings(list(df_tmp[column])).numpy())

        num_embeddings = df_batch_embeddings.shape[1]
        columns = ["emb_" + str(i) for i in range(512)]
        df_tmp[columns] = df_batch_embeddings

        list_dfs.append(df_tmp)
        batch_num = batch_num + 1
        index = index + part

    #concatinate batches into single dataset
    df_emb = pd.concat(list_dfs)

    return df_emb

In [5]:
df_train['noun_phrases'] = df_train['noun_phrases'].str[2:-2]
df_train['noun_phrases'] = df_train['noun_phrases'].str.lower().str.split("', '")
df_train['noun_phrases'].head()

0    [rise, big emerging economy, china, india, steady march, globalisation, surge, number, people, business, tourism, result, demand, visa, unpreceden...
1    [pfizer, commitment, corporate social responsibility csr, drugs giant talk, responsibility, society, world, access, product, work, ngos, global he...
2    [week, federal reserve, interest rate, time, year, world, central bank, rate, recent year, long spell, course, chart, outcome, americas rate rise,...
3    [cruise line, wave, year, nearly, holiday, sea, result, december 18th carnival, worlds largest operator, global market, fullyear earning, demand, ...
4    [investors, calendar year, buoyant mood, unexpected event, consensus, respect, view, investor, market price, column, potential surprise, definitio...
Name: noun_phrases, dtype: object

In [6]:
all_NPs = list(df_train['noun_phrases'])
all_NPs = [np for l in all_NPs for np in l if len(np)>0]
all_NPs[:5], len(all_NPs)

(['rise', 'big emerging economy', 'china', 'india', 'steady march'], 1417049)

In [7]:
df_train['list_of_verb_lemmas'].iloc[0]

'[emerging, led, wanting, travel, granted, Upgrade, travel, apply, submit, streamline, scrap]'

In [8]:
df_train['list_of_verb_lemmas'] = df_train['list_of_verb_lemmas'].str[2:-2]
df_train['list_of_verb_lemmas'] = df_train['list_of_verb_lemmas'].str.lower().str.split(", ")
df_train['list_of_verb_lemmas'].head()

0                                                               [merging, led, wanting, travel, granted, upgrade, travel, apply, submit, streamline, scra]
1    [rided, embracing, insists, gain, strengthen, improve, deterred, seeking, intends, shift, domiciled, rejoiced, saved, paid, outraged, promised, im...
2    [aised, ended, celebrate, tried, lift, forced, reverse, cut, help, understand, upgrade, strike, wish, save, spend, try, escape, slashing, encourag...
3      [race, booked, improve, announced, control, demand, peaking, piling, based, got, moving, upgrade, increase, announced, establish, aimed, based, ad]
4    [tart, caught, proved, reflected, like, suggest, judged, betting, expect, upgrade, weakens, having, pushed, tighten, buy, priced, doubt, tighten, ...
Name: list_of_verb_lemmas, dtype: object

In [9]:
all_Vs = list(df_train['list_of_verb_lemmas'])
all_Vs = [v for l in all_Vs for v in l if len(v)>0]
all_Vs[:5], len(all_Vs)

(['merging', 'led', 'wanting', 'travel', 'granted'], 675330)

In [10]:
all_words =  list(set(all_NPs + all_Vs))
len(set(all_words))

419327

In [11]:
df_words = pd.DataFrame({'word': all_words})
df_words.head()

,word
0,alphabets selfdrive division
1,social media website
2,mr trumps announcement
3,mike finn wolfhard
4,skincare business


In [12]:
%%time
#creating word2vec matrix
df_w2v = get_word_embeddings(df_words, column = "word", N_batches=100)
df_w2v.head()

100 batches with 4194 words each
Batch number: 1 out of  100
Batch number: 2 out of  100
Batch number: 3 out of  100
Batch number: 4 out of  100
Batch number: 5 out of  100
Batch number: 6 out of  100
Batch number: 7 out of  100
Batch number: 8 out of  100
Batch number: 9 out of  100
Batch number: 10 out of  100
Batch number: 11 out of  100
Batch number: 12 out of  100
Batch number: 13 out of  100
Batch number: 14 out of  100
Batch number: 15 out of  100
Batch number: 16 out of  100
Batch number: 17 out of  100
Batch number: 18 out of  100
Batch number: 19 out of  100
Batch number: 20 out of  100
Batch number: 21 out of  100
Batch number: 22 out of  100
Batch number: 23 out of  100
Batch number: 24 out of  100
Batch number: 25 out of  100
Batch number: 26 out of  100
Batch number: 27 out of  100
Batch number: 28 out of  100
Batch number: 29 out of  100
Batch number: 30 out of  100
Batch number: 31 out of  100
Batch number: 32 out of  100
Batch number: 33 out of  100
Batch number: 34 ou

,word,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_502,emb_503,emb_504,emb_505,emb_506,emb_507,emb_508,emb_509,emb_510,emb_511
0,alphabets selfdrive division,0.003092,-0.006629,-0.044545,0.018462,0.001877,0.004101,0.034085,0.055933,-0.026835,...,0.045140,0.064447,-0.019442,-0.030245,-0.033326,0.034933,0.047587,-0.008539,-0.012179,0.028646
1,social media website,-0.049114,0.059202,-0.006767,0.001231,0.001985,0.008514,0.003320,-0.089742,0.018635,...,-0.072986,0.014369,-0.049359,-0.097667,0.053639,-0.071498,0.044204,0.016159,0.052809,0.040297
2,mr trumps announcement,-0.016972,-0.048336,0.001735,-0.046476,0.024367,0.052372,0.003326,-0.032017,0.060843,...,-0.022813,-0.007847,0.107963,0.033170,0.039232,0.013165,-0.026472,0.058270,-0.026721,0.019282
3,mike finn wolfhard,0.006570,-0.051756,-0.002061,0.062292,-0.027771,0.000666,0.002715,0.020261,0.114346,...,-0.020548,-0.002933,-0.071563,-0.012376,0.017611,0.009212,-0.027160,0.021744,0.042835,0.046351
4,skincare business,-0.014702,-0.026797,0.013542,0.017857,0.048319,0.030793,0.040447,0.010290,0.007345,...,-0.023250,0.043505,-0.064382,-0.010494,0.053259,-0.003029,-0.021420,-0.036845,0.008559,-0.013873


In [13]:
df_w2v.iloc[::150001]

,word,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_502,emb_503,emb_504,emb_505,emb_506,emb_507,emb_508,emb_509,emb_510,emb_511
0,alphabets selfdrive division,0.003092,-0.006629,-0.044545,0.018462,0.001877,0.004101,0.034085,0.055933,-0.026835,...,0.045140,0.064447,-0.019442,-0.030245,-0.033326,0.034933,0.047587,-0.008539,-0.012179,0.028646
3246,london cnn businessjust,0.053961,-0.052576,0.008602,0.043876,0.041327,0.028829,0.028812,-0.020806,-0.039280,...,0.007992,-0.026983,0.120966,0.000980,-0.054596,0.013822,0.003440,0.067616,-0.001991,0.015005
2299,evening weekend hour,0.025990,0.001921,0.039863,0.023469,0.009234,0.007211,-0.008845,-0.015654,-0.036591,...,0.082280,-0.038906,0.020474,-0.056085,0.097296,-0.023613,-0.005806,-0.028088,0.093063,-0.053632


# Dimensionality reduction 
UMAP https://github.com/lmcinnes/umap

In [14]:
columns = ["emb_" + str(i) for i in range(512)]
embeddings = df_w2v[columns].values
embeddings.shape

(419327, 512)

In [15]:
%%time
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5, 
                            metric='cosine').fit_transform(embeddings)
umap_embeddings.shape

CPU times: user 55min 11s, sys: 1min 27s, total: 56min 38s
Wall time: 6min 4s


(419327, 5)

In [16]:
with open('./transition_files/umap_embeddings.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(umap_embeddings, f, pickle.HIGHEST_PROTOCOL)

# Clustering

In [17]:
hdbscan_cluster = hdbscan.HDBSCAN(min_cluster_size=15,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)

In [18]:
#cluster labels
labels = hdbscan_cluster.labels_
labels.shape

(419327,)

In [19]:
#number of clusters (key-words/phrases)
print("Number of clusters:",labels.max() + 1)

Number of clusters: 4329


In [20]:
with open('./transition_files/hdbscan_cluster.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(hdbscan_cluster, f, pickle.HIGHEST_PROTOCOL)

# Prepare data for LDA

##### get cluster label as most frequent word/phrase of the cluster

In [26]:
df_tmp = df_w2v[['word']].copy()
df_tmp['cluster_number'] = labels
df_tmp.head()

,word,cluster_number
0,alphabets selfdrive division,228
1,social media website,423
2,mr trumps announcement,-1
3,mike finn wolfhard,-1
4,skincare business,4030


In [34]:
df_all_words = pd.DataFrame({'word': list(all_NPs + all_Vs),
                             'cluster_label': list(all_NPs + all_Vs)
                            })
df_all_words = df_all_words.merge(df_tmp, on='word', how='inner')
print(df_all_words.shape)

(2092379, 3)


In [35]:
df_all_words['word_frequency'] = df_all_words.groupby(['cluster_number', 
                                                       'cluster_label'])['word'].transform("count")
df_all_words['word_max_frequency'] = df_all_words.groupby(['cluster_number'])['word_frequency'].transform("max")
df_all_words.iloc[::222220]

,word,cluster_label,cluster_number,word_frequency,word_max_frequency
0,rise,rise,-1,795,9553
222220,detroit,detroit,3572,85,85
444440,president trump,president trump,3234,276,850
666660,hostility,hostility,724,20,20
888880,ink,ink,1037,29,29
1111100,school bus,school bus,3267,23,130
1333320,private new space company,private new space company,-1,1,9553
1555540,pterodactyl,pterodactyl,1880,1,41
1777760,needed,needed,1912,812,2756
1999980,monitors,monitors,2323,40,444


In [36]:
pd.DataFrame(df_all_words.describe(percentiles=[0.01,0.1,0.20,0.3,0.4,0.5,0.75,0.95,0.99])).T

,count,mean,std,min,1%,10%,20%,30%,40%,50%,75%,95%,99%,max
cluster_number,2092379.0,1524.389263,1515.839422,-1.0,-1.0,-1.0,-1.0,-1.0,421.0,1186.0,2933.0,4210.0,4318.0,4328.0
word_frequency,2092379.0,840.679864,2361.341249,1.0,1.0,1.0,3.0,13.0,46.0,122.0,694.0,3290.0,10360.0,20130.0
word_max_frequency,2092379.0,4265.378710,4717.139590,1.0,8.0,62.0,165.0,366.0,655.0,1245.0,9553.0,9553.0,20130.0,20130.0


In [37]:
df_tmp_noise = pd.DataFrame(df_all_words[df_all_words['cluster_number'] == -1])
df_tmp_other = df_all_words[df_all_words['cluster_number'] != -1]
print(len(df_tmp_noise),len(df_tmp_other))
df_tmp_noise.head()

749438 1342941


,word,cluster_label,cluster_number,word_frequency,word_max_frequency
0,rise,rise,-1,795,9553
1,rise,rise,-1,795,9553
2,rise,rise,-1,795,9553
3,rise,rise,-1,795,9553
4,rise,rise,-1,795,9553


In [38]:
df_tmp = df_tmp_other[df_tmp_other['word_max_frequency'] == df_tmp_other['word_frequency']]
df_tmp = df_tmp.groupby('cluster_number')['cluster_label'].last().reset_index()
print(df_tmp.shape)
df_tmp.iloc[::1000]

(4329, 2)


,cluster_number,cluster_label
0,0,tencent
1000,1000,diaper
2000,2000,almond
3000,3000,good day
4000,4000,minnesota


In [39]:
df_word_clusters = df_all_words[['word', 'cluster_number']]
print(df_word_clusters.shape)

df_word_clusters = df_word_clusters.drop_duplicates()
df_word_clusters = df_word_clusters.merge(df_tmp, on='cluster_number', how='left')
df_word_clusters['cluster_label'] = df_word_clusters['cluster_label'].fillna("noise")

print(df_word_clusters.shape)
df_word_clusters.iloc[::222222].T

(2092379, 2)
(419327, 3)


,0,222222
word,rise,um hand
cluster_number,-1,2655
cluster_label,noise,hand


In [40]:
s = df_word_clusters.groupby('cluster_label')['word'].count()
s.describe()

count      4330.000000
mean         96.842263
std        2538.397253
min          15.000000
25%          23.000000
50%          36.000000
75%          65.000000
max      166821.000000
Name: word, dtype: float64

In [41]:
s.tail()

cluster_label
zika          109
zombie         36
zoo            27
zoom          150
zuckerberg     88
Name: word, dtype: int64

In [42]:
df_word_clusters[df_word_clusters['cluster_label'] == "zombie"]

,word,cluster_number,cluster_label
24011,zombie,313,zombie
24016,corporate zombie,313,zombie
24022,zombie firmscompanie,313,zombie
76572,zombie outbreak,313,zombie
176015,zombie virus,313,zombie
182615,zombie star,313,zombie
193144,zombie infectionsincluding prevention,313,zombie
193150,zombie paper,313,zombie
193152,actual fictional zombie literature,313,zombie
199134,zombie army,313,zombie


In [43]:
df_word_clusters.to_csv('./transition_files/word_cluster_label.csv', index=False)

***
# Replace text words with their cluster names (KeyWords)

In [44]:
df_train.columns

Index(['date', 'author', 'title', 'url', 'section', 'publication',
       'first_10_sents', 'list_of_first_10_sents', 'list_of_verb_lemmas',
       'noun_phrases', 'list_of_nouns', 'list_of_lemmas', 'ID',
       'group_level_1', 'group_level_2', 'group_level_3'],
      dtype='object')

In [45]:
df_train['all_words'] = df_train['list_of_verb_lemmas'] + df_train['noun_phrases']

In [47]:
word_cluster_label_dict =dict(zip(df_word_clusters['word'], df_word_clusters['cluster_label']))

In [49]:
df_train['all_key_words'] = df_train['all_words'].apply(lambda Wlist: 
                                                        [word_cluster_label_dict[w] for w in  Wlist
                                                                                    if w in word_cluster_label_dict
                                                        ])
df_train[['all_key_words', 'all_words']].head()

,all_key_words,all_words
0,"[making, led, noise, travel, noise, upgrade, travel, noise, noise, noise, noise, noise, economic, noise, noise, noise, noise, surge, number, peopl...","[merging, led, wanting, travel, granted, upgrade, travel, apply, submit, streamline, scra, rise, big emerging economy, china, india, steady march,..."
1,"[noise, noise, noise, gain, raised, noise, prevent, noise, noise, shift, noise, excited, noise, noise, noise, promise, noise, letting, noise, tryi...","[rided, embracing, insists, gain, strengthen, improve, deterred, seeking, intends, shift, domiciled, rejoiced, saved, paid, outraged, promised, im..."
2,"[noise, end, noise, trying, lift, noise, reverse, noise, help, understand, upgrade, strike, wish, save, noise, trying, escape, noise, noise, pulle...","[aised, ended, celebrate, tried, lift, forced, reverse, cut, help, understand, upgrade, strike, wish, save, spend, try, escape, slashing, encourag..."
3,"[race, reserved, noise, announced, control, demand, noise, noise, noise, noise, moving, upgrade, noise, announced, making, goal, noise, ad, cruise...","[race, booked, improve, announced, control, demand, peaking, piling, based, got, moving, upgrade, increase, announced, establish, aimed, based, ad..."
4,"[noise, noise, noise, noise, noise, suggest, judgment, betting, expect, upgrade, noise, noise, noise, noise, buy, noise, believes, noise, noise, i...","[tart, caught, proved, reflected, like, suggest, judged, betting, expect, upgrade, weakens, having, pushed, tighten, buy, priced, doubt, tighten, ..."


In [50]:
with open('./transition_files/df_train_for_LDA.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(df_train, f, pickle.HIGHEST_PROTOCOL)